In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib.inline

UsageError: Line magic function `%matplotlib.inline` not found.


In [3]:
df = pd.read_csv(r"https://raw.githubusercontent.com/aniyant/Datasets/main/insurance.csv")

In [6]:
df.shape

(1338, 7)

In [7]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'expenses'], dtype='object')

In [9]:
numerical_columns = [col for col in df.columns if df[col].dtypes != 'O']
numerical_columns

['age', 'bmi', 'children', 'expenses']

In [11]:
categorical_columns = [col for col in df.columns if col not in numerical_columns]
categorical_columns

['sex', 'smoker', 'region']

In [12]:
for col in categorical_columns:
    print(df[col].value_counts())

male      676
female    662
Name: sex, dtype: int64
no     1064
yes     274
Name: smoker, dtype: int64
southeast    364
southwest    325
northwest    325
northeast    324
Name: region, dtype: int64


In [14]:
df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
expenses    float64
dtype: object